# Les Finesses de Python

## Microclub 1er juin 2018

## Philippe Guglielmetti

---

## Pourquoi encore Python ?

> Il ne vaut pas la peine de connaître un langage qui ne modifie pas votre façon de penser la programmation.
> ([Alan Perlis](https://www.drgoulu.com/2008/01/21/perlisismes-les-dictons-informatiques-dalan-perlis/))

* #4 au [TIOBE](https://www.tiobe.com/tiobe-index/) (nombre de développeurs)
* #2 au [Madnight] (https://madnight.github.io/githut/#/issues/2018/1) (activité sur GitHub)
* #1 au [PYPL](http://pypl.github.io/PYPL.html) PopularitY of Programming Language (nombre de tutoriels suivis)



>Python est un langage de programmation objet, multi-paradigme ([wikipedia](https://fr.wikipedia.org/wiki/Python_(langage)))

= on peut programmer :
1. [comme une patate](http://entrian.com/goto/)
2. comme on en a l’habitude (classes, objets, ...)
3. comme on en a pas l’habitude
4. comme ~~un dieu~~ [Guido van Rossum](https://fr.wikipedia.org/wiki/Guido_van_Rossum)

---

* [Librairies](#libs)
* [Objets et Types (Classes)¶](#objs)
* [Introspection](#intro)
* [Les Décorateurs](#deco)
  * [La Memoïsation](#memo)
  * [Le problème de l'arrêt](#stop)
    

<a id="libs"></a>
## Librairies

importons quelques [librairies standard](https://docs.python.org/3/library/) utilisées dans la suite

In [1]:
import logging
import itertools
import functools

<a id="objs"></a>
## Types et Objets (Classes) 

Python utilise un typage:
* FORT : chaque variable désigne un objet dont le type est connu
* DYNAMIQUE : mais une variable peut changer pour désigner un autre objet d'un autre type
* PAS vérifié à la compilation (à moins que...)


In [2]:
a=256
print(a,type(a)) # a désigne un OBJET de type (classe...) int

256 <class 'int'>


In [3]:
print(255+1 == a, 255+1 is a) # == compare les valeurs, is compare les objets
print(a+1 == 257, a+1 is 257) # une idée de ce qui se passe ???

True True
True False


In [4]:
langue_au_chat='! nohtyP ne sinifédérp stejbo sed tnos 652 à 5- ed sreitne sel'
''.join(reversed(langue_au_chat))

'les entiers de -5 à 256 sont des objets prédéfinis en Python !'

In [5]:
s="a = "+str(a) # le str est obligatoire : pas de conversion implicite
print(s,type(s))

a = 256 <class 'str'>


In [6]:
a=str(a) # a ne change pas de type, il désigne un nouvel objet (nuance...)
print(a,type(a))

256 <class 'str'>


In [7]:
def f(n):
    """factorielle récursive"""
    return 1 if n<2 else n*f(n-1)

print(f,type(f)) # les fonctions sont des objets aussi

<function f at 0x0000016D36E01E18> <class 'function'>


In [8]:
class LateX (str): # on peut dériver une classe de n'importe quel type
    
    def __init__(self, s=''): # constructeur
        self = str(s)     # utilise la méthode d'assignation de la classe str
        
    def _repr_latex_(self): # affichage dans Jupyter Notebook
        return r'$%s$'%self
    
    # une méthode est juste un champ de type fonction 
    # les "magic methods" permettent de redéfinir les opérateurs
    __add__=lambda self,right:LateX('{%s}+{%s}'%(self, right))  
    __sub__=lambda self,right:LateX('{%s}-{%s}'%(self, right)) 
    __mul__=lambda self,right:LateX('{%s}{%s}'%(self, right))
    __div__=lambda self,right:LateX('\\frac{%s}{%s}'%(self, right)) 
    __truediv__= __div__ # nécessaire pour / au lieu de //
    
(LateX(2)*'x'+1)/2

'\\frac{{{2}{x}}+{1}}{2}'

In [9]:
# on peut ajouter/modifier dynamiquement des méthodes à une classe
# puisque les méthodes sont aussi des objets !
# (oui c'est très dangereux...)

def puissance_en_latex(self,right): # self n'est pas un mot réservé
    return LateX('{%s}^{%s}'%(self, right))
    
LateX.__pow__=puissance_en_latex

LateX('\\pi')*LateX('r')**2

'{\\pi}{{r}^{2}}'

<a id="intro"></a>
## Introspection
Les champs+méthodes des objets peuvent être examinés.
Et comme tout est objet ...

In [31]:
print(dir(LateX)) # tous les membres de la classe (beaucoup sont hérités de str)

['__add__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__new__', '__pow__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__weakref__', '_repr_latex_', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'tra

In [32]:
print(dir(None)) # membres de l'objet None ?

['__bool__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [11]:
def f(n):
    """factorielle récursive"""
    return 1 if n<2 else n*f(n-1)

print(dir(f)) # membres de l'objet fonction

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [12]:
f.__doc__ # pratique pour écrire des générateurs de doc ...

'factorielle récursive'

In [13]:
import inspect # https://docs.python.org/3/library/inspect.html
inspect.getsourcelines(f)

(['def f(n):\n',
  '    """factorielle récursive"""\n',
  '    return 1 if n<2 else n*f(n-1)\n'],
 1)

In [14]:
f.__code__ # intéressant ... on y reviendra !

<code object f at 0x0000016D36E3FC90, file "<ipython-input-11-2ce22efde7fc>", line 1>

In [52]:
import math # un module est aussi un objet !

for name in math.__dict__: # traverse le dictionnaire de tout ce qui est défini dans le module
    f=math.__dict__[name]
    if callable(f):       # si l'objet a une méthode __call__, on peut l'appeler
        try:
            try:
                print(name+'()=',f())
                continue
            except TypeError:
                pass
            try:
                print(name+'(1)=',f(1))
                continue
            except TypeError:
                pass
            print(name+'(1,2)=',f(1,2))
        except:
            print(name+'(???)')

__loader__()= <_frozen_importlib.BuiltinImporter object at 0x0000016D4E607278>
acos(1)= 0.0
acosh(1)= 0.0
asin(1)= 1.5707963267948966
asinh(1)= 0.8813735870195429
atan(1)= 0.7853981633974483
atan2(1,2)= 0.4636476090008061
atanh(???)
ceil(1)= 1
copysign(1,2)= 1.0
cos(1)= 0.5403023058681398
cosh(1)= 1.5430806348152437
degrees(1)= 57.29577951308232
erf(1)= 0.842700792949715
erfc(1)= 0.157299207050285
exp(1)= 2.718281828459045
expm1(1)= 1.718281828459045
fabs(1)= 1.0
factorial(1)= 1
floor(1)= 1
fmod(1,2)= 1.0
frexp(1)= (0.5, 1)
fsum(???)
gamma(1)= 1.0
gcd(1,2)= 1
hypot(1,2)= 2.23606797749979
isclose(1,2)= False
isfinite(1)= True
isinf(1)= False
isnan(1)= False
ldexp(1,2)= 4.0
lgamma(1)= 0.0
log(1)= 0.0
log1p(1)= 0.6931471805599453
log10(1)= 0.0
log2(1)= 0.0
modf(1)= (0.0, 1.0)
pow(1,2)= 1.0
radians(1)= 0.017453292519943295
sin(1)= 0.8414709848078965
sinh(1)= 1.1752011936438014
sqrt(1)= 1.0
tan(1)= 1.5574077246549023
tanh(1)= 0.7615941559557649
trunc(1)= 1


<a id="deco"></a>
## Les Décorateurs


Un décorateur est une fonction qui modifie une fonction en l'enveloppant.
C'est un [patron de conception](https://fr.wikipedia.org/wiki/Patron_de_conception)

"Gang of four" (GOF) Erich Gamma, Richard Helm, Ralph Johnson, John Vlissides 
“Design Patterns - Catalogue de modèles de conceptions réutilisables” 
Vuibert, 1999, 490 p 

<a id="debug"></a>
### Gestion du niveau de log pour debug

In [15]:
import logging # librairie standard pour loguer

logger = logging.getLogger()
logger.setLevel(logging.INFO)

logging.error("un message d'erreur s'affiche")
logging.warning("un message d'avertissement s'affiche")
logging.info("un message d'info s'affiche")
logging.debug("mais un message de debug est inférieur au level INFO")

ERROR:root:un message d'erreur s'affiche
INFO:root:un message d'info s'affiche


In [16]:
def fonction():
    """cette fonction logue:
    * son propre nom en INFO
    * et sa doc en DEBUG
    """
    logging.info(fonction.__name__)
    def logdoc(doc):
        for s in doc.split('\n'):
            logging.debug(s)
    logdoc(fonction.__doc__)
       
fonction() # ne logue que le nom puisqu'on est au level INFO

INFO:root:fonction


In [17]:
from Goulib import decorators # mes décorateurs préférés

@decorators.debug # passe en level DEBUG et ajoute des INFO à l'entrée et à la sortie
def fonction():
    """cette fonction logue:
    * son propre nom
    * et sa doc
    """
    logging.debug(fonction.__name__)
    def logdoc(doc):
        for s in doc.split('\n'):
            logging.debug(s)
    logdoc(fonction.__doc__)
    
fonction()

INFO:root:Entering fonction
DEBUG:root:fonction
DEBUG:root:cette fonction logue:
DEBUG:root:    * son propre nom
DEBUG:root:    * et sa doc
DEBUG:root:    
INFO:root:Exiting fonction


In [18]:
from Goulib import decorators # mes décorateurs préférés

@decorators.debug # passe en level DEBUG et ajoute des INFO à l'entrée et à la sortie
def fonction():
    """cette fonction logue:
    * son propre nom
    * et sa doc
    """
    logging.debug(fonction.__name__)
    @decorators.nodebug # revenir en INFO pour réduire la taille du log
    def logdoc(doc):
        for s in doc.split('\n'):
            logging.debug(s)
    logdoc(fonction.__doc__)
    
fonction()

INFO:root:Entering fonction
DEBUG:root:fonction
INFO:root:Exiting fonction


<a id="timeit"></a>
### Timeit

écrivons un décorateur pour chronométrer une fonction

In [19]:
import functools # librairie standard

def timeit(func):
    @functools.wraps(func) # un décorateur qui simplifie l'écriture de décorateurs...
    def wrapper(*args, **kwds):
        import time
        t=time.time()
        f_result = func(*args, **kwds)
        t=time.time()-t
        params=', '.join(map(repr,args))
        logger.info('%s(%s) took %f seconds'%(func.__name__,params,t)) 
        return f_result
    return wrapper

In [20]:
def fib(n):
    """une très mauvaise implémentation de la suite de Fibonacci"""
    if n < 2:
      return 1
    return fib(n-1) + fib(n-2)

@timeit
def tfib(n): return fib(n)

[tfib(i) for i in range(0,40,5)]

INFO:root:tfib(0) took 0.000000 seconds
INFO:root:tfib(5) took 0.000000 seconds
INFO:root:tfib(10) took 0.000000 seconds
INFO:root:tfib(15) took 0.000000 seconds
INFO:root:tfib(20) took 0.002010 seconds
INFO:root:tfib(25) took 0.025988 seconds
INFO:root:tfib(30) took 0.270069 seconds
INFO:root:tfib(35) took 2.893746 seconds


[1, 8, 89, 987, 10946, 121393, 1346269, 14930352]

<a id="memo"></a>
### La [Memoïsation](https://fr.wikipedia.org/wiki/M%C3%A9mo%C3%AFsation)

Patron de conception qui accélère les appels répétés à une fonction en mémorisant les résultats en fonction des paramètres

In [21]:
def memoize(obj): # disponible dans Goulib.decorators
    cache = obj.cache = {} # simple dict. il existe des implémentations à mémoire limitée
    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer

In [22]:
@memoize
def mfib(n):
    """la même très mauvaise implémentation de la suite de Fibonacci"""
    if n < 2:
      return 1
    return mfib(n-1) + mfib(n-2)

@timeit
def tfib(n): return mfib(n)

[tfib(i) for i in range(0,40,5)]

INFO:root:tfib(0) took 0.000000 seconds
INFO:root:tfib(5) took 0.000000 seconds
INFO:root:tfib(10) took 0.000000 seconds
INFO:root:tfib(15) took 0.000000 seconds
INFO:root:tfib(20) took 0.000000 seconds
INFO:root:tfib(25) took 0.000000 seconds
INFO:root:tfib(30) took 0.000000 seconds
INFO:root:tfib(35) took 0.000000 seconds


[1, 8, 89, 987, 10946, 121393, 1346269, 14930352]

In [23]:
tfib(500)

INFO:root:tfib(500) took 0.001996 seconds


225591516161936330872512695036072072046011324913758190588638866418474627738686883405015987052796968498626

<a id="stop"></a>
### Le Timeout

In [24]:
t=3
i,n = 0,0
@decorators.timeout(t) # implémentation interrompant la thread
def fibloop():
    global i,n
    while True: # boucle infinie sans le décorateur ...
        i=i+1
        n=fib(i)
        
try:
    fibloop()
except decorators.TimeoutError:
    print('en %d secondes, on peut calculer %d termes'%(t,i))
    print('le dernier est',n)

en 3 secondes, on peut calculer 34 termes
le dernier est 5702887


In [25]:
import itertools
#implémentation au niveau de l'itérateur de boucle
# dommage, pas de @itimeout for ... 
try:
    for i in decorators.itimeout(itertools.count(),t):
        n=mfib(i)
except decorators.TimeoutError:
    print('en %d secondes, on peut calculer %d termes'%(t,i))
    print('le dernier est',n)

en 3 secondes, on peut calculer 200277 termes
le dernier est ...


<a id="deco2"></a>
### Hey mais alors ...

In [90]:
class Valeur:
    def __init__(self,v):
        self.v=v
        self.__name__=type(v).__name__
    @property
    def double(self):
        return self.v*2
    
    def type(self):
        return self.__name__
    
    @classmethod
    def classe(self): # self désigne ici la classe !
        return self.__name__
    
    @staticmethod
    def statique():
        return 'statique'
        
v=Valeur(2)
print(v.classe(), v.type(), v.v, ", double =",v.double, v.statique())

Valeur int 2 , double = 4 statique


... Oui ! certains décorateurs sont prédéfinis et étendent le langage !

## programmation fonctionnelle

In [26]:
for i in range(10):
    print(f(i), end=" ")

1 1 2 6 24 120 720 5040 40320 362880 

In [27]:
r=range(10)
print(r)
print(_ for _ in r)
print([_ for _ in r])

range(0, 10)
<generator object <genexpr> at 0x0000016D37E21C50>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [28]:
res=list(map(f,r))
res

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

In [29]:
import operator
from functools import reduce # était prédéfini en python 2.7
reduce(operator.add,res)


409114